# Scikit-Learn IRIS Model using jsonData

 * Wrap a scikit-learn python model for use as a prediction microservice in seldon-core
   * Run locally on Docker to test
   * Deploy on seldon-core running on minikube
 
## Dependencies

 * [Helm](https://github.com/kubernetes/helm)
 * [Minikube](https://github.com/kubernetes/minikube)
 * [S2I](https://github.com/openshift/source-to-image)

```bash
pip install sklearn
pip install seldon-core
```

## Train locally
 

In [5]:
import numpy as np
import os
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.externals import joblib
from sklearn import datasets

def main():
    clf = LogisticRegression()
    p = Pipeline([('clf', clf)])
    print('Training model...')
    p.fit(X, y)
    print('Model trained!')

    filename_p = 'IrisClassifier.sav'
    print('Saving model in %s' % filename_p)
    joblib.dump(p, filename_p)
    print('Model saved!')
    
if __name__ == "__main__":
    print('Loading iris data set...')
    iris = datasets.load_iris()
    X, y = iris.data, iris.target
    print('Dataset loaded!')
    main()

Loading iris data set...
Dataset loaded!
Training model...
Model trained!
Saving model in IrisClassifier.sav
Model saved!


## REST test

Wrap model using s2i

In [7]:
!s2i build . seldonio/seldon-core-s2i-python3:0.13 sklearn-iris-multipart-formdata:0.1

---> Installing application source...
---> Installing dependencies ...
Looking in links: /whl
You should consider upgrading via the 'pip install --upgrade pip' command.
Build completed successfully


Serve model

In [2]:
!docker run --name "iris_predictor" -d --rm -p 5000:5000 sklearn-iris-multipart-formdata:0.1

415bcc5b2b75e0f8917941dcde830568e30a9074a4faded219d03e1e5ad0bbc8


In [8]:
%%bash

#
# Send some random features to model
#

curl -sg http://localhost:5000/predict -F jsonData='{
        "some_data": {
            "names": [
                "sepal_length",
                "sepal_width",
                "petal_length",
                "petal_width"
            ],
            "some_ndarray": [
                [
                    7.233,
                    4.652,
                    7.39,
                    0.324
                ]
            ]
        }
    }'

{"data":{"names":["t:0","t:1","t:2"],"tensor":{"shape":[1,3],"values":[0.0011809149535247096,0.7867455841528151,0.21207350089366028]}},"meta":{}}


Stop serving model

In [5]:
!docker rm iris_predictor --force

iris_predictor


## Test using Minikube


In [2]:
!minikube start --memory 4096

😄  minikube v1.1.1 on darwin (amd64)
💡  Tip: Use 'minikube start -p <name>' to create a new cluster, or 'minikube delete' to delete this one.
🔄  Restarting existing virtualbox VM for "minikube" ...
⌛  Waiting for SSH access ...
🐳  Configuring environment for Kubernetes v1.14.3 on Docker 18.09.6
🔄  Relaunching Kubernetes v1.14.3 using kubeadm ... 
⌛  Verifying: apiserver proxy etcd scheduler controller dns
🏄  Done! kubectl is now configured to use "minikube"


In [3]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

clusterrolebinding.rbac.authorization.k8s.io "kube-system-cluster-admin" created


In [4]:
!helm init

$HELM_HOME has been configured at /Users/lkuma37/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.

Please note: by default, Tiller is deployed with an insecure 'allow unauthenticated users' policy.
To prevent this, run `helm init` with the --tiller-tls-verify flag.
For more information on securing your installation see: https://docs.helm.sh/using_helm/#securing-your-helm-installation
Happy Helming!


In [5]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

Waiting for rollout to finish: 0 of 1 updated replicas are available...
deployment "tiller-deploy" successfully rolled out


In [55]:
#!helm delete --purge seldon-core
!helm install ../../../helm-charts/seldon-core-operator --name seldon-core --set usageMetrics.enabled=true   --namespace seldon-system

NAME:   seldon-core
LAST DEPLOYED: Fri Aug  2 00:26:15 2019
NAMESPACE: seldon-system
STATUS: DEPLOYED

RESOURCES:
==> v1/ClusterRole
NAME                          AGE
seldon-operator-manager-role  0s

==> v1/ClusterRoleBinding
NAME                                 AGE
seldon-operator-manager-rolebinding  0s

==> v1/ConfigMap
NAME                     DATA  AGE
seldon-config            1     1s
seldon-spartakus-config  1     1s

==> v1/Pod(related)
NAME                                        READY  STATUS             RESTARTS  AGE
seldon-operator-controller-manager-0        0/1    ContainerCreating  0         0s
seldon-spartakus-volunteer-df68b9d48-h2qbb  0/1    ContainerCreating  0         0s

==> v1/Secret
NAME                                   TYPE    DATA  AGE
seldon-operator-webhook-server-secret  Opaque  0     1s

==> v1/Service
NAME                                        TYPE       CLUSTER-IP      EXTERNAL-IP  PORT(S)  AGE
seldon-operator-controller-manager-service  ClusterIP  10.1

In [49]:
!kubectl rollout status deploy/seldon-controller-manager -n seldon-system

partitioned roll out complete: 1 new pods have been updated...


## Setup Ingress


In [56]:
#!helm delete --purge ambassador
!helm install stable/ambassador --name ambassador --set crds.keep=false --set image.tag=0.40.2

NAME:   ambassador
LAST DEPLOYED: Fri Aug  2 00:27:11 2019
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/Deployment
NAME        READY  UP-TO-DATE  AVAILABLE  AGE
ambassador  0/3    3           0          0s

==> v1/Pod(related)
NAME                        READY  STATUS             RESTARTS  AGE
ambassador-d7c4cb5f9-frgcx  0/1    ContainerCreating  0         0s
ambassador-d7c4cb5f9-jzb58  0/1    ContainerCreating  0         0s
ambassador-d7c4cb5f9-n4lp7  0/1    ContainerCreating  0         0s

==> v1/Service
NAME               TYPE          CLUSTER-IP     EXTERNAL-IP  PORT(S)                     AGE
ambassador         LoadBalancer  10.103.254.55  <pending>    80:32306/TCP,443:32061/TCP  0s
ambassador-admins  ClusterIP     10.108.204.21  <none>       8877/TCP                    0s

==> v1/ServiceAccount
NAME        SECRETS  AGE
ambassador  1        0s

==> v1beta1/ClusterRole
NAME        AGE
ambassador  0s

==> v1beta1/ClusterRoleBinding
NAME        AGE
ambassador  0s


NOTES:
C

In [9]:
!kubectl rollout status deployment.apps/ambassador

Waiting for rollout to finish: 0 of 3 updated replicas are available...
Waiting for rollout to finish: 1 of 3 updated replicas are available...
Waiting for rollout to finish: 2 of 3 updated replicas are available...
deployment "ambassador" successfully rolled out


## Wrap Model and Test

In [14]:
!eval $(minikube docker-env) && s2i build . seldonio/seldon-core-s2i-python3:0.13 sklearn-iris-multipart-formdata:0.1

---> Installing application source...
---> Installing dependencies ...
Looking in links: /whl
You should consider upgrading via the 'pip install --upgrade pip' command.
Build completed successfully


In [57]:
!eval $(minikube docker-env)
!kubectl create -f sklearn_iris_multipart_formdata_deployment.json

seldondeployment.machinelearning.seldon.io/seldon-deployment-example created


In [40]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=sklearn-iris-deployment-seldon-deployment-example --no-headers=true|cut -d' ' -f1)


deployment "sklearn-iris-deployment-sklearn-iris-predictor-dde2107" successfully rolled out


### Test by sending prediction calls

In [63]:
%%bash

PAYLOAD=$(
cat <<EOF
{
        "some_data": {
            "names": [
                "sepal_length",
                "sepal_width",
                "petal_length",
                "petal_width"
            ],
            "some_ndarray": [
                [
                    7.233,
                    4.652,
                    7.39,
                    0.324
                ]
            ]
        }
}
EOF
)

NAMESPACE_NAME=default
DEPLOYMENT_NAME=seldon-deployment-example

HOST=$(minikube ip)
PORT=$(kubectl get svc ambassador -o jsonpath='{.spec.ports[0].nodePort}')

set -x
curl -s -X POST -H 'Content-Type:multipart/form-data' \
    -F jsonData="${PAYLOAD}" \
    "http://${HOST}:${PORT}/seldon/${NAMESPACE_NAME}/${DEPLOYMENT_NAME}/api/v0.1/predictions" 
set +x

{
  "meta": {
    "puid": "ms6idff33vpjprj52k0go9g501",
    "tags": {
    },
    "routing": {
    },
    "requestPath": {
      "sklearn-iris-classifier": "sklearn-iris-multipart-formdata:latest"
    },
    "metrics": []
  },
  "data": {
    "names": ["t:0", "t:1", "t:2"],
    "tensor": {
      "shape": [1, 3],
      "values": [0.0011809149535247096, 0.7867455841528151, 0.21207350089366028]
    }
  }
}

+ curl -s -X POST -H Content-Type:multipart/form-data -F 'jsonData={
        "some_data": {
            "names": [
                "sepal_length",
                "sepal_width",
                "petal_length",
                "petal_width"
            ],
            "some_ndarray": [
                [
                    7.233,
                    4.652,
                    7.39,
                    0.324
                ]
            ]
        }
}' http://192.168.99.128:32306/seldon/default/seldon-deployment-example/api/v0.1/predictions
+ set +x


In [ ]:
!minikube delete